In [1]:
import numpy as np
import pandas as pd
import nltk
import re

In [2]:
hotel = pd.read_csv('uk_hotel.csv', index_col = 0)
hotel.head()

,Unnamed: 0.1,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Lower_Neg_Review,Lower_Pos_Review,Negative_Review_Clean,Positive_Review_Clean
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/17,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3,52.360576,4.915968,my room was dirty and i was afraid to walk ba...,great location in nice surroundings the bar a...,my room was dirty and i was afraid to walk ba...,great location in nice surroundings the bar a...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/17/17,7.7,Hotel Arena,United Kingdom,Cleaner did not change our sheet and duvet ev...,33,1403,...,6,4.6,"[' Leisure trip ', ' Group ', ' Duplex Twin Ro...",17,52.360576,4.915968,cleaner did not change our sheet and duvet ev...,the room is spacious and bright the hotel is ...,cleaner did not change our sheet and duvet ev...,the room is spacious and bright the hotel is ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/17/17,7.7,Hotel Arena,United Kingdom,Apart from the price for the brekfast Everyth...,11,1403,...,1,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double...",17,52.360576,4.915968,apart from the price for the brekfast everyth...,good location set in a lovely park friendly s...,apart from the price for the brekfast everyth...,good location set in a lovely park friendly s...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/7/17,7.7,Hotel Arena,United Kingdom,Nothing all great,5,1403,...,2,10.0,"[' Leisure trip ', ' Group ', ' Duplex Double ...",27,52.360576,4.915968,nothing all great,rooms were stunningly decorated and really sp...,nothing all great,rooms were stunningly decorated and really sp...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/6/17,7.7,Hotel Arena,United Kingdom,The floor in my room was filfy dirty Very bas...,28,1403,...,7,4.6,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",28,52.360576,4.915968,the floor in my room was filfy dirty very bas...,comfy bed good location,the floor in my room was filfy dirty very bas...,comfy bed good location


In [3]:
hotel.shape

(124881, 22)

In [4]:
## https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk_tag_to_wordnet_tag(nltk_tag):
    '''convert nltk tag to wordnet tag'''
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    '''perform lemmatization based on part of speech'''
    # tokenize the sentence and find the POS tag for each token (get part of speech)
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    # tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence) 

In [5]:
def remove(lines: pd.Series):
    '''tokenize each line and remove the stopwords'''
    l = []
    for line in lines:
        words = nltk.word_tokenize(line)
        new_line = ''
        for word in words:
            if word not in stopword_list:
                new_line += word + ' '
        l.append(new_line)
    return l

In [6]:
from nltk.corpus import stopwords
print(set(stopwords.words('english')))

{"hasn't", 'here', "aren't", 'wouldn', 're', 'how', 'hers', 'has', 'the', 'you', 'don', 'shan', 'which', 'aren', 'once', 'from', "you'll", "it's", 'under', 'only', 'further', "mightn't", 'yourself', 'had', 'are', 'does', "you'd", 'over', 'too', 'of', 'd', 'yourselves', 'but', 'off', 'down', 'during', 'such', 'she', "doesn't", "shan't", 'and', 'before', 'then', 'mustn', 'been', 'again', 'y', "isn't", 'below', 'a', 's', 'as', 'mightn', "won't", 'it', 'he', 'to', 'up', 'this', 'into', 'at', 'hasn', 'they', 'ours', 'was', 'i', 'haven', 'what', 'wasn', 'him', "shouldn't", 'itself', 'other', 've', 'between', 'there', 'his', 'whom', 'few', 'is', "wouldn't", 'yours', "mustn't", 'theirs', 'if', 'ain', 'by', 'each', 'them', 'most', 'won', 'your', "wasn't", 'or', 'now', 'an', 'when', 'for', 'those', 'her', 'myself', 'after', 'where', 'having', 'nor', 'himself', 'than', 'doesn', "didn't", 'about', "should've", 'some', 'more', 'needn', 'that', 'until', 'both', 'with', 'should', 'weren', 'we', 'bein

In [7]:
from gensim.parsing.preprocessing import STOPWORDS
print(STOPWORDS)

frozenset({'here', 'has', 'inc', 'under', 'further', 'never', 'yourself', 'hereafter', 'does', 'too', 'say', 'yourselves', 'get', 'hundred', 'wherein', 'whereas', 'down', 'otherwise', 'fill', 'then', 'call', 'someone', 'everywhere', 'at', 'ours', 'was', 'ten', 'fifteen', 'nine', 'i', 'him', 'neither', 'co', 'between', 'sometimes', 'con', 'regarding', 'yours', 'if', 'fire', 'made', 'across', 'km', 'make', 'various', 'seems', 'name', 'than', 'himself', 'doesn', 'used', 'sometime', 'some', 'both', 'until', 'we', 'around', 'thereafter', 'every', 'side', 'although', 'my', 'any', 'amount', 'empty', 'namely', 'however', 'will', 'thru', 'de', 'not', 'show', 'same', 'themselves', 'three', 'really', 'through', 'next', 'became', 'thereupon', 'hence', 'front', 're', 'hers', 'whoever', 'computer', 'already', 'must', 'are', 'using', 'over', 'sixty', 'off', 'often', 'she', 'and', 'before', 'done', 'forty', 'well', 'nowhere', 'up', 'this', 'amongst', 'describe', 'whence', 'latterly', 'whenever', 'mean

In [8]:
# customized stopword list by adding common but not useful words

stopword_list = set(list(stopwords.words('english'))+list(STOPWORDS)).\
union({ 'would',  'really', '-' , 'even', 'hotel', 'UK', 'U.K.', 'well', 'able', 'try', 'tried', 'definitely', 'actual', 
       'actually', 'addional', 'additionally', 'absolutely', 'anymore', 'apparently', 'basically', 'received',
       'completely', 'extremely', 'generally', 'hello', 'anymore', 'definitely', 'finally', 'honestly', 
       'just', 'lol', 'literally', 'item', 'oh','ok','okay','r','thank','thanks','thing','yes','yeah',
       'maybe', 'mean', 'mention', 'mind', 'need', 'normal', 'normally', 'opinion', 'overall', 'people', 'person',
       'personally', 'probably', 'recieved', 'saw', 'simply', 'somewhat', 'think', 'thought', 'totally', 'truly',
       'usually', 'want', 'wow', 'www', 'wo', 'slight', 'slightly', 'allow','apply','arrive','ask','ate','available',
       'away','bring','care','carry', 'certainly','com','come','consider','decide','discover','especially','exactly',
       'expect', 'feel','felt','general','gp','href','http','let','make','agree', 'believe', 'super','little',
      'thankyou', 'exceedingly', 'everything', 'mmmm', 'unfortunately'})

# keep privatives as they are useful when adding sequence into consideration in N-gram language model

keep_stopwords = ['not', "don't", 'no', "doesn't", 'doesn', 'cannot', "mightn't", "can't", 'without', "didn't", 
                 "haven't", 'cant', "hasn't", "weren't", 'wouldn', 'neither', "wouldn't", "isn't", 'didn', "mustn't",
                 "shouldn't", "needn't", 'weren', "couldn't", 'don', 'aren', "aren't", 
                 'couldn', 'couldnt', 'hadn', 'haven', 'shouldn', 'wasn', "wasn't", 'won', "won't"]
stopword_list.difference_update(keep_stopwords)

## Positive Review

In [9]:
# lemmatization

lemm_positive = []

for i in hotel['Positive_Review_Clean']:
    lemm_positive.append(lemmatize_sentence(i))
    
hotel['lemm_positive'] = lemm_positive

In [10]:
hotel['positive_remove_stopwords'] = remove(hotel['lemm_positive'])

In [11]:
# regex grouping

clean_text = [] 
replacement = [
    (r'comf(?:y|ortable)', 'comfortable'),
    (r'min(?:ute)?', 'minute'),
    (r'helpful(?:l)?', 'helpful'), 
    (r'welcom(?:e|ing|inuteg)?', 'welcome')
]

for line in hotel['positive_remove_stopwords']:
    for pattern, word in replacement:  
        line = re.sub(pattern, word, line)
    clean_text.append(line)

hotel['positive_regex'] = clean_text

### Further handling

In [12]:
# Find the most common words

from collections import Counter
import nltk
counter=Counter()
for review in hotel['positive_regex']:
    words = nltk.word_tokenize(review)  
    for word in words:
        counter[word]+=1

In [13]:
wordcount=list(counter.keys())

In [14]:
len(wordcount)

17428

In [15]:
commonword=pd.read_csv('20k.txt')
common=commonword.values.reshape(-1,).tolist()

In [16]:
wrongspell=[]
for word in wordcount:
    if word in common:
        continue
    else:
        wrongspell.append(word)

In [17]:
pd.DataFrame(wrongspell).to_excel('wrong_spell_pos.xlsx')

In [18]:
# regex grouping wrong spelling

clean_text = [] 
replacement = [
    (r'\b(\w+)?\d+(\w+)?\b','_NUMBER_'),
    (r'\bdis(?:s)?ap(?:p)?(?:o)?(?:i)?(?:n)?t(?:ed|ing)?\b','disappoint'),
    (r'surp(?:p|r)?ise(?:d|ing)?', 'surprise'),
    (r'\beveveryone\b', 'everyone'),
    (r'\babosultely|absolutly\b', 'absolutely'),
    (r'\bunfortunatly\b', 'unfortunately'),
    (r'\benviroment|invironment\b', 'environment'),
    (r'\bbadroom\b', 'bedroom'),
    (r'\bwelcomering|wecominuteg\b', 'welcoming'),
    (r'\bo(?:o)?stpark', 'oosterpark'),
    (r'\bne(?:s|ss)?presso(?:s)?|expresso|coffe(?:e)?\b', 'espresso'),
    (r'\btrafic\b', 'traffic'),
    (r'\bflavour\b', 'flavor'),
    (r'\balresdy\b', 'already'),
    (r'\bsmokinh\b', 'smoking'),
    (r'\brefebishment|refurbisging\b', 'refurbishment'),
    (r'\bhe(?:a)?l(?:p|o)?(?:e)?ful(?:l)?\b', 'helpful'),
    (r'\bextre(?:a)?(?:m)?(?:em)?ly|extreamly|extremley|extrememy\b', 'extremely'),
    (r'\bprofesional\b', 'proffesional'),
    (r'\bsuper{1,}b\b', 'superb'),
    (r'\bdelious\b', 'delicious'),
    (r'\bavaiable\b', 'available'),
    (r'\bchocolat\b', 'chocolate'),
    (r'\babsoulty|absoulte\b', 'absolute'),
    (r'\bpleasently\b', 'pleasantly'),
    (r'\bbriliant\b', 'brilliant'),
    (r'\bsomehwere\b', 'somewhere'),
    (r'\bawsome\b', 'awesome'),
    (r'\bminutei\b', 'mini'),
    (r'\bminuteibar\b', 'minibar'),
    (r'\bminuteimalist\b', 'minimalist'),
    (r'\bhelpfulnes\b', 'helpfulness'),
    (r'\bhelpfulnes\b', 'helpfulness'),
    (r'\btasy\b', 'tasty'),
    (r'\bluxuorios\b', 'luxurious'),
    (r'\bbedr(?:r|o)om(?:s)?\b', 'bedroom'),
    (r'\byoghurt\b', 'yogurt'),
    (r'\blovley\b', 'lovely'),
    (r'\bwvery\b', 'very'),
    (r'\bvarierty\b', 'variety'),
    (r'\bdiscrepcy\b', 'discrepancy'),
    (r'\bwonderfulf\b', 'wonderful'),
    (r'\boveral{1,}\b', 'overalll'),
    (r'\barrrived\b', 'arrived'),
    (r'\buncomftable\b', 'uncomfortable'),
    (r'\befficent\b', 'efficient'),
    (r'\btallkiing\b', 'talking'),
    (r'\bevrything\b', 'everything'),
    (r'\bfsmily\b', 'smily'),
    (r'\bftrsnsport\b', 'transport'),
    (r'\btouble\b', 'trouble'),
    (r'\bspacey\b', 'spacy'),
    (r'\bswimminutegpool\b', 'swimmingpool'),
    (r'\barrrival\b', 'arrival'),
    (r'\bso{1,}\b', 'so'),
    (r'\bwelll\b', 'well'),
    (r'\bfanatastic\b', 'fantastic'),
    (r'\bintially\b', 'initially'),
    (r'\bresepction\b', 'reception'),
    (r'\bproffesionally\b', 'professionally'),
    (r'\bhaply\b', 'happy'),
    (r'\bwslking\b', 'walking'),
    (r'\bsomethings\b', 'something'),
    (r'\bminuteimise|minuteiute\b', 'minute'),
    (r'\bathmosphe\b', 'athmosphere'),
    (r'\bimpresssively\b', 'impressively'),
    (r'\bneeed\b', 'need'),
    (r'\bconsiderng\b', 'considering'),
    (r'\bclosenes\b', 'closeness'),
    (r'\bminuteiature\b', 'miniature'),
    (r'\bcleaness\b', 'cleanness'),
    (r'\bthouight\b', 'thought'),
    (r'\bstunnig\b', 'stunning'),
    (r'\barcatecture\b', 'architecture'),
    (r'\bmusuem\b', 'museum'),
    (r'\bamamzing|amazing{0,}|anazing\b', 'amazing'),
    (r'\bprobaly\b', 'probably'),
    (r'\bthrougout\b', 'througout'),
    (r'\bpoplar\b', 'popular'),
    (r'\bofferred\b', 'offer'),
    (r'\bdeterminutee\b', 'determine'),
    (r'\bfunjy\b', 'funny'),
    (r'\bcostumer\b', 'customer'),
    (r'\bprefereed\b', 'prefer'),
    (r'\bminuteinal\b', 'mininal'),
    (r'\binmediately\b', 'immediately'),
    (r'\bhamman\b', 'hammam'),
    (r'\bbettter\b', 'better'),
    (r'\boffen\b', 'often'),
    (r'\bgoed\b', 'good'),
    (r'\blitte\b', 'little'),
    (r'\bnatutal\b', 'natural'),
    (r'\bpoilite\b', 'polite'),
    (r'\beledely\b', 'elderly'),
    (r'\bmemnbers\b', 'members'),
    (r'\bperfeect\b', 'perfect'),
    (r'\bsrvice\b', 'service'),
    (r'\bpowerfull\b', 'powerful'),
    (r'\bunderwhelminuteg\b', 'underwhelming'),
    (r'\bcheep\b', 'cheap'),
    (r'\bparticuarly\b', 'particularly'),
    (r'\bmahine\b', 'machine'),
    (r'\bplastc\b', 'plastic'),
    (r'\bperfecto\b', 'perfect'),
    (r'\bbr{1,}illiant\b', 'brilliant'),
    (r'\bieverywhere\b', 'everywhere'),
    (r'\bgreate|greating\b', 'greet'),
    (r'\bmorden|mordern|moderness\b', 'modern'),
    (r'\bconviently|convenieny|convienient|convieniaces|convinence|convieniaces\b', 'convenient'),
    (r'\bspacous|spaceous\b', 'spacious'),
    (r'\bpositon|possition|positon\b', 'position'),
    (r'\batmosphere{0,}|atmerphere|atmerphere\b', 'atmosphere'),
    (r'\bhelpfuly|helpf\b', 'helpful'),
    (r'\bacomadating|accomadating|accomdation|accomadate\b', 'accommodate'),
    (r'\brecomend|reccomend|recomand|reccommend|recomened|recccomend|reccomendations\b', 'recommend'),
    (r'\bcleanessnes|clenliness|cleanlness\b', 'cleanness'),
    (r'\bbeutiful|beatiful|beuatiful|beautifull|beatufuil|beatifull|beauitful\b', 'beautiful'),
    (r'\bcomfey|comphy|comfty|comftable|confortable|confy|counfy|cmfy|confort|compfy|comforatble|\
    comfprtable|comforable|vomfortable|comferable|comfortsble|comftorbale|comfotable|comfie|comfortabl|confttable|\
    comfortablew|conformable\b', 'comfortable'),
    (r'\bllocation|locatiom|lcation|locarion|loction|locution|iocation|lovation|kocation\b', 'location'),
    (r'\bbarthroom|barhroom|bathrooom|bagroom|bathtroom\b', 'bathroom'),
    (r'\bqueit|quinet\b', 'quiet'),
    (r'\breminuteiscent\b', 'reminiscent'),
    (r'\bexcellnt|excellant|eccellent|ecellent|excellen|excelleny\b', 'excellent'),
    (r'\bthebreakfast|brekfast|beakfast|breakfadt|beeakfast|breskfast|breakfasy|brakefeast\b', 'breakfast'),
    (r'\brestaraunt|restorant|restraunts|resteraunt|resteraunt|resturant|resataurant|reastraunts|restaurante|\
     restsurant|restruants|restaraunts|restuarants|resturants|restraunt|restaurent|resturaunt|testaurant\b', \
     'restaurant'),
    (r'\boverwhelminuteg|overwhelminutegly|overwhelm|overwhelminutegly\b', 'overwhelming'),
    (r'\bwestminutester|westminuteister|westmister|westminuteter\b', 'westminster'),
    (r'\bfacilties|facilities\b', 'facility'),
    (r'\bmony|mmmm\b', 'many'),
    (r'\bveiw|viiew\b', 'view'),
    (r'\bdefinatly|defintely|definetly|definetley|deffinately|definateley|definetely\b', 'definitely'),
    (r'\bexellent|rxcellent|ecellent\b', 'excellet'),
    (r'\bfrendly|fiendly|freindly|friandly|frielndly|friendlly|friendless|friendeliness|friwndly|feiendly|\
    frindly\b', 'friendly'),
    (r'\beverthing|wverything|everything{0,}|everytjing|evrerthing|everuthing\b', 'everything'),
    (r'\brecepti0nist|receptionest|receptionnist|recptionist\b', 'receptionist'),
    (r'\bfantasics|fantatic|fantatsic\b', 'fantastic'),
    (r'\battentative|attentetive\b', 'attentive'),
    (r'\bprossecco|procesco\b', 'prosseco'),
    (r'\bunerground|undergraound|ununderground|nearunderground\b', 'underground')
]

for line in hotel['positive_regex']:
    for pattern, word in replacement:  
        line = re.sub(pattern, word, line)
    clean_text.append(line)

hotel['positive_regex'] = clean_text

In [19]:
# remove stopwords again

hotel['positive_regex'] = remove(hotel['positive_regex'])

In [20]:
# regex to reduce overlaps in topic modeling

clean_text = [] 
replacement = [
    (r'\bhelpfulness\b', 'helpful'),
    (r'\bdisppointed|disppoints|disappointing\b', 'disappoint'),
    (r'\bmodernise|modernize\b', 'modern'),
    (r'\bpricey\b', 'expensive'),
    (r'\beatery\b', 'restaurant'),
    (r'\bpoliteness\b', 'polite'),
    (r'\bspacy\b', 'spacious'),
    (r'\brefurb(?:ed|ing)?|refurbish(?:ing)?|refurnish\b', 'refurbish'),
    (r'\breceptionist(?:s)?|receptive\b', 'reception'),
    (r'\bfriendlyness\b', 'friendly'),
    (r'\bcouldn(?:t)?\b', 'couldnt'),
    (r'\bwouldn(?:t)?\b', 'couldnt'),
    (r'\battentive(?:ly)?\b', 'attentive'),
    (r'\baccomodation(?:s)?\b', 'accomodation'),
    (r'\bclean(?:ness)?\b', 'clean'),
    (r'\bprofessional(?:ly)?\b', 'professional'),
    (r'\bcelebratory|congratulatory\b', 'celebrate'),
    (r'\bconvenient(?:ly)?|convenience\b', 'convenient'),
    (r'\bcomfort(?:ing|ness|able{0,})?|comfortably|comfortability\b', 'comfortable'),
    (r'\bsurprise(?:ing|d)?\b', 'surprise'),
    (r'\bgood|excellent|great|nice|wonderful|marvellous|amazing|fantastic|perfect(ion)?|awesome|\
    stunningly|brilliantly|unbelievably|amaze|amazing|stun|excellently|exceptionallly|brilliant|impressively|\
    outstandingly|incredibly\b', 'good'),
    (r'\bgoodtt\b', 'good')
]

for line in hotel['positive_regex']:
    for pattern, word in replacement:  
        line = re.sub(pattern, word, line)
    clean_text.append(line)

hotel['positive_regex_further'] = clean_text

### TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer   

vectorizer_3 = TfidfVectorizer(ngram_range=(3,3), 
                               token_pattern=r'\b\w\w+\b',
                               min_df=5, 
                               # max_df=0.1,
                               # max_features= 10000
                               ) 

In [22]:
corpus_positive = list(hotel['positive_regex_further'].values)

X_further = vectorizer_3.fit_transform(corpus_positive)
terms = vectorizer_3.get_feature_names()
tf_idf_positive_3 = pd.DataFrame(X_further.toarray().transpose(), index=terms)

In [23]:
tf_idf_positive_3

,0,1,2,3,4,5,6,7,8,9,...,124871,124872,124873,124874,124875,124876,124877,124878,124879,124880
_number_ _number_ _number_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_number_ _number_ adult,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_number_ _number_ bed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_number_ _number_ breakfast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
_number_ _number_ central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
year old grandson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
year old son,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
young lady check,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
young lady reception,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
tf_idf_positive_3 = tf_idf_positive_3.sum(axis = 1) 
score_positive_3 = pd.DataFrame(tf_idf_positive_3, columns=["score"])
score_positive_3.sort_values(by="score", ascending=False, inplace=True)
top = score_positive_3.head(300)
top

,score
staff friendly helpful,1155.073150
friendly helpful staff,742.693326
good location good,606.846644
comfortable bed good,523.510775
staff helpful friendly,517.824689
...,...
helpful bed comfortable,49.712669
location good easy,49.671534
extra comfortable bed,49.358209
good size bed,48.714237


In [26]:
pd.DataFrame(top.index).to_excel('tfidf_pos.xlsx')

In [27]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=2)
W_positive = nmf.fit_transform(X_further)
H_positive = nmf.components_
print(f"Original shape of X positive is {X_further.shape}")
print(f"Decomposed W positive matrix is {W_positive.shape}") 
print(f"Decomposed H positice matrix is {H_positive.shape}")

/Users/liujinglei/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Original shape of X positive is (124881, 14695)
Decomposed W positive matrix is (124881, 2)
Decomposed H positice matrix is (2, 14695)


In [28]:
from typing import List
import numpy as np

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens] 
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
        print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

In [29]:
positive_tf_idf = pd.DataFrame(X_further.toarray(), columns=terms)

In [30]:
positive_tf_idf_drop = positive_tf_idf.drop(['friendly helpful staff', 'good location friendly', 'location friendly staff', 'friendly helpful stay', \
    'staff good location', 'helpful staff good', 'staff good modern', 'location friendly helpful', 'friendly helpful time', 'helpful staff lot', 'good location goodly', \
        'friendly helpful trouble', 'helpful staff not', 'staff good professional', 'location good base', 'staff friendly oblige', \
            'friendly staff help', 'staff good roof', 'good friendly staff', 'staff good selection', 'staff good soon',
                'staff good recommend', 'good staff member', 'helpful staff speak', 'staff good rooftop', 'good staff not', 'location good decor', 'staff good size'], axis = 1)

In [31]:
get_top_tf_idf_tokens_for_topic(H_positive, positive_tf_idf_drop.columns.tolist(), 5)

TOPIC 0

staff friendly offer (27.4%)

good staff leave (2.1%)

friendly helpful room (1.6%)

friendly helpful good (1.1%)

comfortable staff friendly (1.1%)

TOPIC 1

friendly helpful upgrade (21.6%)

good location heart (3.2%)

helpful staff leave (3.2%)

staff good quiet (2.9%)

location good beautiful (2.8%)



In [32]:
import numpy as np
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
    print("=" * 50)

In [33]:
get_top_documents_for_each_topic(W_positive, hotel['positive_regex_further'].tolist(), num_docs=5)

# some examples from the documents showing why there is too much overlap between the topics

Topic 0
100.0% staff friendly helpful 
100.0% staff friendly helpful tiny room creatively appoint good bathroom food breakfast limit good quality bar facility lobby lovely touch 
100.0% staff friendly helpful 
100.0% staff friendly helpful pleasant 
100.0% staff friendly helpful 
Topic 1
100.0% friendly helpful staff 
100.0% friendly helpful staff 
100.0% good standard friendly helpful staff 
100.0% cleanliness friendly helpful staff 
100.0% friendly helpful staff 


many reviewers mention about both staff and location as well as the room in their positive reviews

## Negative Review

In [34]:
# lemmatization

lemm_negative = []

for i in hotel['Negative_Review_Clean']:
    lemm_negative.append(lemmatize_sentence(i))
    
hotel['lemm_negative'] = lemm_negative

In [35]:
hotel['negative_remove_stopwords'] = remove(hotel['lemm_negative'])

In [36]:
# regex grouping

clean_text = [] 
replacement = [
    (r'\b(?:little|bit)\b', 'little'),
    (r'\bair(?:\scondition(?:er|ing)?)?(?:\sunit)?\b','air_conditioner'),
    (r'\bnot\sinclude\b','not include'),
    (r'\bcentre\b','center'),
    (r'\bmachine\b', 'facility'),
    (r'\bwi\sfi\b','wifi'),
    (r'\bcomfy\b','comfortable'),
    (r'\bfirm\b','hard'),
    (r'\b(?:tea\s)?(?:coffee\s)?(?:making\s)?facility\b','drink_facility'),
    (r'\btea\scoffee\b','tea_coffee')
]

for line in hotel['negative_remove_stopwords']:
    for pattern, word in replacement:  
        line = re.sub(pattern, word, line)
    clean_text.append(line)

hotel['negative_regex'] = clean_text

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer   

vectorizer_3 = TfidfVectorizer(ngram_range=(3,3), 
                               token_pattern=r'\b\w\w+\b', 
                               min_df=3,
                               # max_features= 6000,  
                               stop_words=stopword_list) 

In [38]:
corpus_negative = list(hotel['negative_regex'].values)

X_neg = vectorizer_3.fit_transform(corpus_negative)
terms = vectorizer_3.get_feature_names()
tf_idf_negative_3 = pd.DataFrame(X_neg.toarray().transpose(), index=terms)

/Users/liujinglei/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['hadn', 'uk'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [39]:
tf_idf_negative_3 = tf_idf_negative_3.sum(axis = 1) 
score_negative_3 = pd.DataFrame(tf_idf_negative_3, columns=["score"])
score_negative_3.sort_values(by="score", ascending=False, inplace=True)
top_neg = score_negative_3.head(300)
top_neg

,score
no drink_facility room,216.678175
breakfast not include,148.014629
no hot water,119.329142
bed not comfortable,110.894606
air_conditioner didn work,99.853642
...,...
price mini bar,18.595718
no big deal,18.524104
not _number_ _number_,18.467287
wait _number_ hour,18.444752


In [40]:
len(score_negative_3)

21539

In [41]:
pd.DataFrame(top_neg.index).to_excel('tfidf_neg.xlsx')

In [42]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=3)
W_neg = nmf.fit_transform(X_neg)
H_neg = nmf.components_
print(f"Original shape of X negative is {X_neg.shape}")
print(f"Decomposed W negative matrix is {W_neg.shape}") # verify shape
print(f"Decomposed H negative matrix is {H_neg.shape}")

/Users/liujinglei/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Original shape of X negative is (124881, 21539)
Decomposed W negative matrix is (124881, 3)
Decomposed H negative matrix is (3, 21539)


In [43]:
neg_tf_idf = pd.DataFrame(X_neg.toarray(), columns=terms)

In [44]:
get_top_tf_idf_tokens_for_topic(H_neg,neg_tf_idf.columns.tolist(),5)

TOPIC 0

no drink_facility room (60.0%)

drink_facility room no (1.0%)

room no drink_facility (0.7%)

drink_facility room room (0.6%)

disappoint no drink_facility (0.5%)

TOPIC 1

breakfast not include (43.1%)

not include price (8.9%)

not include expensive (2.3%)

not include room (1.5%)

not include book (1.3%)

TOPIC 2

bed not comfortable (36.7%)

small bed not (1.6%)

sofa bed not (1.5%)

not comfortable room (1.5%)

room small bed (1.4%)



### further handling

In [46]:
# Find the most common words
from collections import Counter
import nltk
from nltk.corpus import stopwords

counter=Counter()
for review in hotel['negative_regex']:
    words = nltk.word_tokenize(review)        
    for word in words:
        counter[word]+=1

In [47]:
wordcount=list(counter.keys())

type(wordcount)
len(wordcount)

19726

In [48]:
commonword=pd.read_csv('20k.txt')
common=commonword.values.reshape(-1,).tolist()

In [49]:
wrongspell_neg=[]
for word in wordcount:
    if word in common:
        continue
    else:
        wrongspell_neg.append(word)

In [50]:
pd.DataFrame(wrongspell_neg).to_csv('wrong_spell_neg.csv')

In [51]:
# regex grouping
import re
clean_text = [] 
replacement = [
    (r'\b(\w+)?\d+(\w+)?\b','_NUMBER_'),
    (r'\bfilfy\b','filthy'),
    (r'\bbre(aa)?kfast\b', 'breakfast'),
    (r'\baak\b','ask'),
    (r'\baap\b','as soon as possible'),
    (r'\babso(u)?l(o)?ut(e)?l(e)?y\b','absolutely'),
    (r'\baccom(m)?(?:a|o)?(d)?(?:e|a)?t(?:e(d)?|ing|ion|iin)\b','accommadation')
]

for line in hotel['negative_regex']:
    for pattern, word in replacement:  
        line = re.sub(pattern, word, line)
    clean_text.append(line)

hotel['negative_regex'] = clean_text

In [52]:
hotel.to_csv('hotel_processing_new.csv')